In [9]:
import sys
sys.path.append('../src')

import network as nw
import torch
import generate_kvadraturen as gk
import bus
import road as rd
import traffic_lights as tl
import junction as jn
import numpy as np
import loading_json as load
import json

In [13]:
x = torch.tensor(0.4, requires_grad=True)

In [35]:
y1 = x**2
y1.backward()
print(x.grad)

tensor(17.6000)


In [2]:
f = open("../kvadraturen_networks/network_1.json")
data = json.load(f)
f.close()
T = data["T"]
N = data["N"]
speed_limits = data["speed_limits"] # Nested list
control_points = data["control_points"] # Nested list
cycle_times = data["cycle_times"] # Nested list

In [3]:
f = open("../kvadraturen_networks/config_1_1.json")
config = json.load(f)
f.close()

In [4]:
bus_network = gk.generate_kvadraturen_from_config(T, N, speed_limits, control_points, cycle_times,
                                                  config)

In [5]:
# bus_network = gk.generate_kvadraturen_roundabout_w_params(T, N, speed_limits, control_points, cycle_times,
#                                                           track_grad=False)

In [6]:
# T = 30
# bus_network = gk.generate_kvadraturen_w_bus(T, track_grad=False)

In [5]:
# with torch.autograd.set_detect_anomaly(True):
#     densities, queues, bus_lengths, bus_delays = bus_network.solve_cons_law()

densities, queues, bus_lengths, bus_delays = bus_network.solve_cons_law()

Bus 2 reached bus stop 0 at time 43.75009536743164, should wait for 29.593318939208984 seconds


In [6]:
final_time = list(densities[0])[-1]
for i, road in enumerate(bus_network.roads):
    print(f"Road {road.id} has a queue length {queues[i][final_time]}")

Road v_strand_1fw has a queue length 0.0
Road v_strand_2fw has a queue length 0.0
Road v_strand_3fw has a queue length 0.0
Road v_strand_4fw has a queue length 0.0
Road v_strand_5fw has a queue length 0.0
Road v_strand_6fw has a queue length 0.0
Road v_strand_1bw has a queue length 0.0
Road v_strand_2bw has a queue length 0.0
Road v_strand_3bw has a queue length 0.0
Road v_strand_4bw has a queue length 0.0
Road v_strand_5bw has a queue length 0.0
Road v_strand_6bw has a queue length 0.0
Road h_w_1 has a queue length 0.210833340883255
Road h_w_2 has a queue length 0.0
Road tollbod_2fw has a queue length 0.0
Road tollbod_1bw has a queue length 0.0
Road tollbod_2bw has a queue length 0.0
Road elvegata_fw has a queue length 0.0
Road elvegata_bw has a queue length 0.0
Road dronning_1fw has a queue length 0.0
Road dronning_2fw has a queue length 0.0
Road dronning_3fw has a queue length 0.0
Road dronning_1bw has a queue length 0.0
Road dronning_2bw has a queue length 0.0
Road dronning_3bw has

In [10]:
for i, lengths in bus_lengths.items():
    print("-----------------------------------------------")
    print(f"Lengths for bus {i+1}")
    for t, l in lengths.items():
        print(f"t =  {t}, l = {l}")
    print("-----------------------------------------------")
    print()

-----------------------------------------------
Lengths for bus 1
t =  0, l = 0.0
t =  0.4687500298023224, l = 6.523437976837158
t =  0.7970176339149475, l = 11.076778411865234
t =  1.1005127429962158, l = 15.248891830444336
t =  1.393972635269165, l = 19.266063690185547
t =  1.6826436519622803, l = 23.226852416992188
t =  1.968900203704834, l = 27.152116775512695
t =  2.253920793533325, l = 30.928743362426758
t =  2.5383081436157227, l = 34.70039367675781
t =  2.8223721981048584, l = 38.46962356567383
t =  3.106271982192993, l = 42.206504821777344
t =  3.3900885581970215, l = 45.93730926513672
t =  3.673863410949707, l = 49.675514221191406
t =  3.9576172828674316, l = 53.422691345214844
t =  4.241360664367676, l = 58.013916015625
t =  4.52509880065918, l = 62.60088348388672
t =  4.808834075927734, l = 67.18533325195312
t =  5.0, l = 70.26654815673828
t =  5.283733367919922, l = 74.841064453125
t =  5.5674662590026855, l = 79.41905212402344
t =  5.851199150085449, l = 83.98477172851562

Testing why bus 1 stops at 23.23 metres for such a long time:

23.23 metres is at the end of vs_mainline_3 and vs_mainline_4

In [7]:
bus_1 = bus_network.busses[0]
bus_1.get_road_position(bus_network)

(41, tensor(0.9291), 42)

In [9]:
print(bus_network.roads[42].id)

vs_mainline_4


In [10]:
bus_1.get_road_id()

('vs_mainline_3', tensor(23.2269), 'vs_mainline_4')

In [20]:
t = torch.tensor(25.0)
dt = torch.tensor(0.3)
slowdown_factors = [torch.ones(road.N_internal+1) for road in bus_network.roads]
slowdown_factors, slowing_idx = bus_network.update_position_of_bus(bus_1, dt, t, slowdown_factors)

In [21]:
slowing_idx

41

In [22]:
print(bus_network.roads[41].rho)
print(bus_network.roads[41].rho)


tensor([0.1748, 0.1748, 0.1745, 0.1739, 0.1730, 0.1730])
tensor([0.1748, 0.1748, 0.1745, 0.1739, 0.1730, 0.1730])


In [23]:
print(slowing_idx)
for i, road in enumerate(bus_network.roads):
    if i == slowing_idx:
        print(f"Slowdown factors on road {road.id}:")
        print(slowdown_factors[i])
        print()


41
Slowdown factors on road vs_mainline_3:
tensor([1., 1., 1.])



Something looks strange here - investigate what happens by manually doing the steps in update_position_of_bus:

In [29]:
road_id, length, next_id = bus_1.get_road_id()
print(road_id, length, next_id)
print(road_id == "")

vs_mainline_3 tensor(23.2269) vs_mainline_4
False


In [32]:
i, road = bus_network.get_road(road_id)
print(i, road.id)
slowing_idx = None

41 vs_mainline_3


In [34]:
slowdown_, bus_started_ = bus_1.get_slowdown_factor(slowdown_factors[41].clone(), road_id,
                                                      length, road)
print(slowdown_, bus_started_)

tensor([1., 1., 1.]) True


There are no stops on this road, so no slowing down should happen!

In [40]:
print(bus_started_)
slowing_idx = 41

True


In [43]:
# Looking for opposite lane:
opposite_road = None
if road.id[-2:] == "fw" or road.id[-2:] == "bw":
    if i > 0: 
        if bus_network.roads[i-1].id[:-2] == road.id[:-2]:
            opposite_road = bus_network.roads[i-1]
    if i < len(bus_network.roads)-1:
        if bus_network.roads[i+1].id[:-2] == road.id[:-2]:
            opposite_road = bus_network.roads[i+1]
print(opposite_road)

None


As expected there are no opposing lanes...

In [44]:
print(opposite_road is not None)

False


Slowdown factors calculated correctly...

In [47]:
new_length = length / road.L
new_length # length travelled on road
speed = road.get_speed_updated(new_length, dt) * road.L
speed # Speed in m/s on road


tensor(11.5015)

Speed looks okay

In [53]:
activation = torch.tensor(1.0)
if next_id == "" or length + speed * dt < road.L*road.b:
    print("Bus not close to junction")
else:
    print("Bus close to junction")

Bus close to junction


In [55]:
for j in bus_network.junctions:
    check, activ_ = j.get_activation(t, road_id, next_id)
    print(check, activ_)
    if check:
        activation = activ_
        break

print(activation)

False 0.0
False 0.0
False 0.0
False 0.0
False 0.0
False 0.0
False 0.0
False 0.0
False 0.0
False 0.0
False 0.0
False 0.0
False 0.0
False 0.0
tensor(1.)


In [58]:
for road in j.roads:
    print(road.id)

festning_3fw
festning_4fw
festning_3bw
festning_4bw


In [59]:
if activation >= 0.5:
    # Bus can cross junction
    speed = torch.minimum(speed, j.get_speed(t, road_id, next_id))
    print(speed)
print(road_id, next_id)

tensor(4.8247e-26)
vs_mainline_3 vs_mainline_4


In [62]:
idx_1 = -1
idx_2 = -1
for i, road in enumerate(j.road_in):
    if road.id == road_id:
        idx_1 = i
        break
for i, road in enumerate(j.road_out):
    if road.id == next_id:
        idx_2 = i
        break
print(idx_1, idx_2)

-1 -1


testing outgoing speed on lundsbro - looks to be way too fast

In [36]:
import json
print("Loading results...")
f = open("../results/kvadraturen_500_temp_opt_internal.json")
data = json.load(f)
f.close()
densities = data[0]
queues = data[1]
bus_lengths = data[2]
bus_delays = data[3]

Loading results...


In [44]:
lengths = bus_lengths['2']

In [45]:
for t, length in lengths.items():
    print(f"Length at time {t}: {length}")

Length at time 0.0: 0.0
Length at time 0.4687500298023224: 0.0
Length at time 0.7970176339149475: 0.0
Length at time 1.1005127429962158: 0.0
Length at time 1.393972635269165: 0.0
Length at time 1.6826436519622803: 0.0
Length at time 1.968900203704834: 0.0
Length at time 2.253920793533325: 0.0
Length at time 2.5383081436157227: 0.0
Length at time 2.8223721981048584: 0.0
Length at time 3.106271982192993: 0.0
Length at time 3.3900885581970215: 0.0
Length at time 3.673863410949707: 0.0
Length at time 3.9576172828674316: 0.0
Length at time 4.241360664367676: 0.0
Length at time 4.52509880065918: 0.0
Length at time 4.808834075927734: 0.0
Length at time 5.0: 0.0
Length at time 5.283733367919922: 0.0
Length at time 5.5674662590026855: 0.0
Length at time 5.851199150085449: 0.0
Length at time 6.134932041168213: 0.0
Length at time 6.418664932250977: 0.0
Length at time 6.70239782333374: 0.0
Length at time 6.986130714416504: 0.0
Length at time 7.269863128662109: 0.0
Length at time 7.553595542907715:

In [25]:
for t, length in lengths.items():
    print(f"Length at time {t}: {length}")

Length at time 0.0: 0.0
Length at time 0.4687500298023224: 6.523437976837158
Length at time 0.7970176339149475: 11.076778411865234
Length at time 1.1005127429962158: 15.248891830444336
Length at time 1.393972635269165: 19.266063690185547
Length at time 1.6826436519622803: 23.226852416992188
Length at time 1.968900203704834: 23.226852416992188
Length at time 2.253920793533325: 23.226852416992188
Length at time 2.5383081436157227: 23.226852416992188
Length at time 2.8223721981048584: 23.226852416992188
Length at time 3.106271982192993: 23.226852416992188
Length at time 3.3900885581970215: 23.226852416992188
Length at time 3.673863410949707: 23.226852416992188
Length at time 3.9576172828674316: 23.226852416992188
Length at time 4.241360664367676: 23.226852416992188
Length at time 4.52509880065918: 23.226852416992188
Length at time 4.808834075927734: 23.226852416992188
Length at time 5.0: 23.226852416992188
Length at time 5.283733367919922: 23.226852416992188
Length at time 5.5674662590026

In [7]:
objective_fnc = torch.tensor(0.0)
for i in range(len(bus_delays)):
    for j in range(len(bus_delays[i])):
        objective_fnc = objective_fnc + bus_delays[i][j]
print(objective_fnc)

tensor(40.0934, grad_fn=<AddBackward0>)


In [8]:
objective_fnc.backward()
speed_limits = bus_network.get_speed_limit_grads()
traffic_lights = bus_network.get_traffic_light_grads()

In [ ]:
59.24377647  80.          80.          65.87917085  57.49176426
  79.26315085  80.          50.13192135  28.42084207  49.62954308
  99.99005596  50.2654001   99.99631058  50.03542892  99.99824851
  50.29059791  99.99172916  49.99478611  99.99989917  70.12932961
  69.99999647  79.33517426  79.80488551  49.68705362  99.98898422
  59.79298455 119.88902983  60.76570415 119.98281134  59.87032718
 119.99935821  79.77154544  99.98828991  79.96011442  98.09888928

In [9]:
speed_limits[4]

0.04000745341181755

In [5]:
times = list(densities[0].keys())
objective_fnc += densities[0][times[-1]][4]

In [5]:
objective_fnc.backward()

In [6]:
bus_network.get_speed_limit_grads()

[tensor(0.0009),
 tensor(3.3250e-25),
 tensor(-1.3810),
 tensor(-0.6805),
 tensor(0.0400),
 tensor(0.0084),
 tensor(-0.6718),
 tensor(1.4207e-20),
 tensor(-1.2328e-09)]

In [7]:
bus_network.get_traffic_light_grads()

[tensor(-2.7601e-24),
 tensor(0.),
 tensor(-4.7805e-21),
 tensor(0.),
 tensor(-1.5719e-20),
 tensor(0.),
 tensor(-4.5082e-20),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(-4.8610e-20),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(1.1688e-22),
 tensor(0.),
 tensor(3.1627e-27),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(7.5706e-21),
 tensor(0.),
 tensor(-9.7813e-19),
 tensor(0.),
 tensor(-3.1493e-18),
 tensor(0.)]

#### Testing speed of bus coming into junction

In [21]:
# Interesting bus: first bus
test_bus = bus_network.busses[0]
test_bus_lengths = bus_lengths[0]

# Interesting roads:
incoming = bus_network.roads[28]
outgoing = bus_network.roads[14]
in_densities = densities[28]
out_densities = densities[14]

# Interesting junction:
jnc = bus_network.junctions[7]

# Interesting time period: 
start_time = torch.tensor(344)
end_time = torch.tensor(366)

In [29]:
# First check densities on roads in the interesting interval:
times = list(in_densities.keys())
for i, t in enumerate(times):
    if start_time < t < end_time:
        next_t = times[i+1]
        dt = next_t - T
        tot_length = test_bus_lengths[t]
        _, length = test_bus.get_road_id_at_length(tot_length)
        new_length = length / incoming.L
        prev, next, pos = incoming.get_node_at_length(new_length)

        print(f"t = {t}")
        print(f"Bus has travelled {tot_length} meters, {length} meters on current road")
        print(f"This is equivalent to internal nodes {prev}, and {next}, with pos {pos}")
        print(f"Incoming densities: ")
        print(in_densities[t])
        print(f"Outgoing densities: ")
        print(out_densities[t])
        print()

t = 344.200927734375
Bus has travelled 644.899169921875 meters, 94.899169921875 meters on current road
This is equivalent to internal nodes 7.0, and 8.0, with pos 7.091933727264404
Incoming densities: 
tensor([2.3842e-07, 2.3842e-07, 2.3842e-07, 2.3842e-07, 2.3859e-07, 2.3994e-07,
        2.4461e-07, 2.5648e-07, 2.6570e-07, 4.3250e-06, 6.9844e-03, 3.0192e-01,
        9.2697e-01, 9.9957e-01, 1.0000e+00, 1.0000e+00], grad_fn=<CopySlices>)
Outgoing densities: 
tensor([1.3129e-10, 1.3129e-10, 4.7617e-11, 1.5330e-11, 4.4689e-12, 1.3288e-12,
        3.9349e-13, 1.1662e-13, 3.4558e-14, 1.0241e-14, 3.0348e-15, 8.9931e-16,
        2.6650e-16, 7.8972e-17, 2.3402e-17, 6.9349e-18, 2.0550e-18, 6.0904e-19,
        1.8058e-19, 5.3407e-20, 1.5374e-20, 4.5237e-21, 2.8527e-21, 5.1930e-21,
        2.0519e-20, 9.4007e-20, 6.3478e-19, 6.3478e-19], grad_fn=<CopySlices>)

t = 344.65093994140625
Bus has travelled 651.149169921875 meters, 101.149169921875 meters on current road
This is equivalent to internal n

In [34]:
# Check when bus
first_bus_lengths = bus_lengths['0']
first_bus = bus_network.busses[0]
times = first_bus_lengths.keys()
for t in times:
    length = first_bus_lengths[t]
    if length > 700:
        print(f"t = {t}, length = {length}")
        print(f"Road id: {first_bus.get_road_id_at_length(length)}")
        break

t = 365.0, length = 700.0109252929688
Road id: ('tollbod_2fw', 0.01092529296875)


In [4]:
# 27.550016403198242, should wait for 29.835927963256836 seconds
# 344.65093994140625 - 365 - Check calculated speeds...

In [4]:
print(bus_delays)

{0: [tensor(202.3306, grad_fn=<AddBackward0>), tensor(199.9107, grad_fn=<AddBackward0>), tensor(242.2030, grad_fn=<AddBackward0>), tensor(205.3318, grad_fn=<AddBackward0>)], 1: [tensor(23.8124, grad_fn=<AddBackward0>), tensor(5.1012, grad_fn=<AddBackward0>), tensor(0., grad_fn=<AddBackward0>), tensor(119.0426, grad_fn=<AddBackward0>)], 2: [tensor(0.), tensor(0.), tensor(0.), tensor(0.)], 3: [tensor(0., grad_fn=<AddBackward0>), tensor(0., grad_fn=<AddBackward0>), tensor(0., grad_fn=<AddBackward0>), tensor(0.)]}


In [5]:
objective_fnc = torch.tensor(0.0)
for i in range(len(bus_delays)):
    for j in range(len(bus_delays[i])):
        objective_fnc = objective_fnc + bus_delays[i][j]
print(objective_fnc)

tensor(997.7323, grad_fn=<AddBackward0>)


In [6]:
objective_fnc.backward()

In [ ]:
adgHSKKL

NameError: name 'adgHSKKL' is not defined

In [ ]:
rho = torch.tensor([0.1, 0.1, 0.2, 0.45, 0.35, 0.234, 0.124, 0.64, 0.24])
slowdown = torch.tensor([0.1, 0.5, 0.7, 0.2, 0.6, 0.1])
pad = 2
n_internal = len(rho) - 2*pad
for i in range(n_internal+1):
    print(rho[-pad-i], rho[-pad-i-1])

internal = torch.flip(rho[pad-1:-pad+1],dims=[0])
avg = (internal[1:] + internal[:-1]) / 2
torch.ones(6) - (1. - slowdown) * (1. - avg) / 2

tensor(0.6400) tensor(0.1240)
tensor(0.1240) tensor(0.2340)
tensor(0.2340) tensor(0.3500)
tensor(0.3500) tensor(0.4500)
tensor(0.4500) tensor(0.2000)
tensor(0.2000) tensor(0.1000)


tensor([0.7219, 0.7947, 0.8938, 0.7600, 0.8650, 0.6175])

In [12]:
n = 4
m = 3
A = torch.ones((n, m))
B = torch.tensor([[1.3, 5.2, 12.4],
                  [24.1, 0.04, 0.5],
                  [12.1, 12.4, 22.2],
                  [1.2, 4.6, 7.8]])
i = 3
j = 1
tot_sum = torch.tensor(0.0)
mask = torch.ones(n, dtype=torch.bool)
mask[i] = False
filtered_tensor = torch.cat((B[:i,j], B[i+1:,j]))
for l in range(n):
    if l != i:
        tot_sum += B[l,j]
second_sum = torch.sum(B[mask, j])

tot_sum
second_sum - tot_sum

tensor(0.)

In [2]:
dict = {i: [0.1 + i] for i in range(3)}
dict

{0: [0.1], 1: [1.1], 2: [2.1]}

In [7]:
densities_loaded = load.convert_from_tensor(densities)
queues_loaded = load.convert_from_tensor(queues)
bus_lengths_loaded = load.convert_from_tensor(bus_lengths)

In [5]:
bus_delays_dict = {0: [float(l.detach()) for l in bus_delays[0]],
                   1: [float(l.detach()) for l in bus_delays[1]],
                   2: [float(l.detach()) for l in bus_delays[2]]}
                #    3: [float(l.detach()) for l in bus_delays[3]]}
                   

In [8]:
# Write above dictionaries to json file
import json
# with open("kvadraturen_results_minimal_1000_variable.json", 'w') as fd:
#     fd.write(json.dumps([densities, queues, bus_lengths, bus_delays_dict]))
with open("kvadraturen_500_temp_opt.json", 'w') as fd:
    fd.write(json.dumps([densities_loaded, queues_loaded, bus_lengths_loaded, bus_delays_dict]))

In [9]:
import json
print("Loading results...")
f = open("kvadraturen_500_temp_opt.json")
data = json.load(f)
f.close()
densities = data[0]
queues = data[1]
bus_lengths = data[2]
bus_delays = data[3]

Loading results...


In [ ]:
import openGLUtilsGif as opgl
opgl.draw_busses_w_densities(bus_network, [bus_fw, bus_bw, bus_fw_2, bus_bw_2], bus_lengths,
                                    densities, output_name="test_roundabout.gif")

Maximum and minimum x points: 7, -0.5
Maximum and minimum y points: -0.4, 8.9
Window created


: 

In [10]:
import json
print("Loading results...")
f = open("../results/kvadraturen_500_temp_opt.json")
data = json.load(f)
f.close()
densities = data[0]
queues = data[1]
bus_lengths = data[2]
bus_delays = data[3]

Loading results...


In [11]:
shortened_densities = {}
for road_id in densities.keys():
    shortened_densities[road_id] = {}
    for t in densities[road_id].keys():
        old_dens = densities[road_id][t]
        new_dens = [old_dens[i] for i in range(2, len(old_dens)-2)]
        shortened_densities[road_id][t] = new_dens        

In [12]:
with open("kvadraturen_500_temp_opt_internal.json", 'w') as fd:
    fd.write(json.dumps([shortened_densities, queues, bus_lengths, bus_delays]))

In [43]:
def short_stick_prob(x, d):
    '''
    Probability of a point x in [0,1] being hit by a stick of length d <= 0.5
    '''
    if x <= d:
        return x / (1 - d)
    elif x <= 1 - d:
        return d / (1 - d)
    else:
        return (1 - x) / (1 - d)
    
def long_stick_prob(x, d):
    '''
    Probability of a point x in [0,1] being hit by a stick of length d > 0.5
    '''
    if x <= 1 - d:
        return x / (1 - d)
    elif x <= d:
        return torch.tensor(1.0)
    else:
        return (1 - x) / (1 - d)
    
def stick_prob(x,d):
    '''
    Probability of a point x in [0,1] being hit by a stick of length d (> 0)
    '''
    if d <= 0.5:
        return short_stick_prob(x,d)
    else:
        return long_stick_prob(x,d)

In [23]:
def trapezoidal_rule(f, a=0.0, b=1.0, n=10):
    '''
    Assuming equidistant spacing
    '''
    dx = (b-a) / n
    xk = torch.linspace(a, b, n+1)
    int = torch.tensor(0.0)
    int += (f(xk[0]) + f(xk[-1]))
    for i in range(1,n):
        int += 2 * f(xk[i])
    int = int * dx / 2
    return int

In [49]:
d_list = [torch.tensor(0.3, requires_grad=True), torch.tensor(0.2, requires_grad=True), torch.tensor(0.7, requires_grad=True)]


# output = trapezoidal_rule(lambda x: 1 - torch.prod(torch.tensor([1-stick_prob(x,d) for d in d_list])), 0, 1, 4)
def temp_func(x):
    integrand = torch.tensor(1.0)
    for d in d_list:
        integrand *= (1-stick_prob(x,d))
    return 1.0 - integrand
# output = trapezoidal_rule(lambda x: 1 - (1-stick_prob(x,d_list[0]))*(1-stick_prob(x,d_list[1]))*(1-stick_prob(x,d_list[2])) , 0, 1, 4)
# output = trapezoidal_rule(temp_func, 0, 1, 4)
# output = trapezoidal_rule(lambda x: 1.0 - torch.prod(torch.stack([(1 - stick_prob(x, d)) for d in d_list])), 0, 1, 4)

# print(output)
temp_func_lambda = lambda x : 1.0 - torch.prod(torch.stack([(1.0 - stick_prob(x, d)) for d in d_list]))
output = trapezoidal_rule(temp_func_lambda, 0, 1, 4)
output

tensor(0.7098, grad_fn=<DivBackward0>)

In [45]:
x = 0.3
print(1.0 - stick_prob(x, d_list[0]), 1.0 - stick_prob(x, d_list[1]), 1.0 - stick_prob(x, d_list[2]))
print(torch.stack([(1.0 - stick_prob(x, d)) for d in d_list]))
print(torch.prod(torch.stack([(1.0 - stick_prob(x, d)) for d in d_list])))


tensor(0.5714, grad_fn=<RsubBackward1>) tensor(0.7500, grad_fn=<RsubBackward1>) tensor(0., grad_fn=<RsubBackward1>)
tensor([0.5714, 0.7500, 0.0000], grad_fn=<StackBackward0>)
tensor(0., grad_fn=<ProdBackward0>)


In [48]:
output.backward()

for d in d_list:
    print(d.grad)

tensor(0.0319)
tensor(0.0837)
tensor(0.6696)


In [ ]:
tensor(0.0319)
tensor(0.0837)
tensor(0.6696)

In [42]:
bus_1 = bus_network.busses[1]

In [43]:
bus_1.get_road_id_at_length(1140.189)

('vs_mainline_2', 15.189000000000078)